<a href="https://colab.research.google.com/github/oluwafemidiakhoa/MLprject/blob/main/SmartDevOps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install openai pandas streamlit docker
!pip install flask flask-restful




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 3.3 MB/s eta 0:00:00


In [38]:
import re
import os
import traceback
import logging
import subprocess
import sys
from openai import OpenAI

# Function to install packages
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install necessary packages
try:
    import flask
    import flask_restful
except ImportError:
    install('flask')
    install('flask-restful')
    import flask
    import flask_restful

# Initialize OpenAI client
# Initialize the OpenAI client
client = OpenAI(api_key=)


# Logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class PlannerService:
    def create_backlog(self):
        backlog = ["Design REST API", "Develop Feature X", "Create Unit Tests"]
        logging.info(f"Planner Service: Created backlog: {backlog}")
        return backlog


class DeveloperService:
    def generate_code(self, task, retry_count=2):
        prompt = f"Generate Python code to {task}. Ensure the code is optimized, modular, and follows best practices."

        for attempt in range(retry_count + 1):
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are an AI software developer."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=300
            )
            code = self.extract_code(response.choices[0].message.content.strip())

            if self.is_code_valid(code):
                logging.info(f"Developer Service: Successfully generated code for task '{task}' on attempt {attempt + 1}:\n{code}")
                return code
            else:
                logging.warning(f"Developer Service: Invalid code detected on attempt {attempt + 1}, retrying...")
                prompt = f"Regenerate the Python code for {task}. The last attempt had issues with code validity. Please ensure the code is complete, properly indented, and follows best practices."

        logging.error("Developer Service: Failed to generate valid Python code after multiple attempts.")
        return ""

    def extract_code(self, raw_response):
        # Search for code blocks
        code_blocks = re.findall(r"```python\n(.*?)\n```", raw_response, re.DOTALL)
        if code_blocks:
            return "\n".join(code_blocks)
        else:
            # Fallback: attempt to find any line that starts with valid Python keywords or structures
            lines = raw_response.splitlines()
            code_lines = [line for line in lines if self.is_valid_python_line(line)]
            if code_lines:
                return "\n".join(code_lines)
            else:
                logging.warning("Developer Service: No valid Python code found.")
                return ""

    def is_valid_python_line(self, line):
        # Simple heuristic to determine if a line looks like Python code
        python_keywords = ["def ", "class ", "import ", "from ", "return ", "print(", "if ", "for ", "while "]
        return any(line.strip().startswith(keyword) for keyword in python_keywords)

    def is_code_valid(self, code):
        """Check if the generated code is valid, e.g., no obvious syntax errors like missing indentation."""
        lines = code.splitlines()
        for i, line in enumerate(lines):
            stripped_line = line.strip()
            if stripped_line.startswith("if ") or stripped_line.startswith("for ") or stripped_line.startswith("while "):
                if i + 1 < len(lines) and lines[i + 1].strip() == "":
                    return False
                if i + 1 < len(lines) and not lines[i + 1].startswith("    "):
                    return False
            if stripped_line.startswith("class ") and ":" in stripped_line:
                if i + 1 < len(lines) and lines[i + 1].strip() == "":
                    return False
                if i + 1 < len(lines) and not lines[i + 1].startswith("    "):
                    return False
        return True


class TesterService:
    def run_tests(self, code):
        logging.info("Tester Service: Running tests...")
        test_results = {}
        if not code.strip():
            logging.error("Tester Service: No code to test.")
            return {'status': 'Failed', 'error': 'No valid code provided.'}

        try:
            exec_globals = {}
            exec(code, exec_globals)
            # Simulate basic test passing by ensuring there's no exception
            test_results['status'] = "Passed"
            logging.info("Tester Service: Tests passed.")
        except Exception as e:
            logging.error(f"Tester Service: Tests failed with error: {traceback.format_exc()}")
            test_results['status'] = "Failed"
            test_results['error'] = str(e)
        return test_results


class DeploymentService:
    def deploy_code(self, code):
        if not code.strip():
            logging.error("Deployment Service: No code to deploy.")
            return {"status": "Failed", "error": "No valid code to deploy."}

        try:
            exec_globals = {}
            exec(code, exec_globals)
            logging.info("Deployment Service: Code executed successfully.")
            return {"status": "Deployed"}
        except Exception as e:
            logging.error(f"Deployment Service: Deployment failed with error: {traceback.format_exc()}")
            return {"status": "Failed", "error": str(e)}


class MonitorService:
    def monitor_application(self):
        logging.info("Monitor Service: Monitoring simulated application.")
        # Placeholder for AI-driven monitoring logic
        # Simulate monitoring by analyzing logs or output
        return {"status": "Monitoring"}


def run_smart_devops():
    # Step 1: Planning
    planner = PlannerService()
    backlog = planner.create_backlog()

    # Step 2: Development and Testing
    developer = DeveloperService()
    tester = TesterService()

    for task in backlog:
        code = developer.generate_code(task)
        if not code:
            print(f"Failed to generate valid code for '{task}'. Skipping...")
            continue

        print(f"Generated code for '{task}':\n{code}")

        # Step 3: Testing
        if "flask" in code and "app.run" in code:
            print(f"Skipping execution of Flask app for task '{task}' as it cannot be run in Colab.")
            test_results = {'status': 'Failed', 'error': 'Flask app cannot be tested in this environment.'}
        else:
            test_results = tester.run_tests(code)

        print(f"Test results for '{task}':", test_results)

        # Step 4: Deployment (only if tests pass)
        if test_results['status'] == "Passed":
            deployment = DeploymentService()
            deploy_info = deployment.deploy_code(code)
            print(f"Deployment info for '{task}':", deploy_info)

            # Step 5: Monitoring
            monitor = MonitorService()
            monitor_status = monitor.monitor_application()
            print(f"Monitoring status for '{task}':", monitor_status)
        else:
            print(f"Skipping deployment for '{task}' due to failed tests.")


# Run the SmartDevOps simulation in Colab
run_smart_devops()


Generated code for 'Design REST API':
from flask import Flask
from flask_restful import Api, Resource, reqparse
class User(Resource):
    def get(self, name):
        for user in users:
                return user, 200
        return "User not found", 404
    def post(self, name):
        for user in users:
                return "User with name {} already exists".format
Test results for 'Design REST API': {'status': 'Passed'}
Deployment info for 'Design REST API': {'status': 'Deployed'}
Monitoring status for 'Design REST API': {'status': 'Monitoring'}
Generated code for 'Develop Feature X':
class Calculator:
    def __init__(self):
        pass

    @staticmethod
    def add(num1, num2):
        return num1 + num2

    def subtract(self, num1, num2):
        return num1 - num2

    def multiply(self, num1, num2):
        return num1 * num2

    def divide(self, num1, num2):
        if num2 != 0:
            return num1 / num2
        else:
            raise ZeroDivisionError('Division